In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from transformers import RobertaTokenizer
import numpy as np
import tensorflow as tf
from transformers import TFRobertaModel
from sklearn.metrics import classification_report, balanced_accuracy_score

In [18]:
df = pd.read_csv('C:\\Users\\thebl\\Documents\\Lomba\\SatDat2024\\FullSenyum-SatriaData2024\\DataSet\\cleaned_final_label.csv')
df

,Unnamed: 0,text,label,stemmed_text,label_angka
0,0,kunjungan prabowo untuk meresmikan menyerahkan...,Sumber Daya Alam,kunjung prabowo untuk resmi serah proyek bantu...,5
1,1,anies tepuk tangan meriah jadi rektor mewajibk...,Politik,anies tepuk tangan riah jadi rektor wajib mata...,0
2,2,emng bener sih pendukung anies juga pendukung ...,Demografi,emng bener sih dukung anies juga dukung prabow...,6
3,3,sewaktu anies bersikap kritis kinerja pak prab...,Politik,waktu anies sikap kritis kerja pak prabowo ang...,0
4,4,anies baswedan harap asn termasuk tni polri pe...,Politik,anies baswedan harap asn masuk tni polri pegan...,0
...,...,...,...,...,...
4578,4578,ngeliat debat kemaren pas prabowo kicep kekira...,Politik,ngeliat debat kemaren pas prabowo kicep kira k...,0
4579,4579,masyarakat yakin prabowo gibran memiliki visi ...,Politik,masyarakat yakin prabowo gibran milik visi jal...,0
4580,4580,imo both are irrational but satu jauh lebih ir...,Ekonomi,imo both are irrational but satu jauh lebih ir...,4
4581,4581,look at that pak ganjar sudah berkecimpung lgi...,Pertahanan dan Keamanan,look at that pak ganjar sudah kecimpung lgisla...,3


In [19]:
df = df.drop(columns=['Unnamed: 0'])

In [20]:
df= df.drop_duplicates(subset='stemmed_text')

In [25]:
df_politik = df[df['label'] == 'Politik']

# Inisialisasi penghitung kemunculan
jumlah_kemunculan = 0

# Iterasi melalui setiap teks yang difilter
for teks in df_politik['stemmed_text']:
    # Pisahkan teks menjadi kata-kata dan hitung kemunculan kata 'ekonomi'
    jumlah_kemunculan += teks.lower().split().count('ekonomi')

print("Jumlah kemunculan kata 'ekonomi' dalam label 'politik':", jumlah_kemunculan)

Jumlah kemunculan kata 'ekonomi' dalam label 'politik': 35


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from transformers import BertTokenizer, TFBertModel
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, balanced_accuracy_score

# Assuming df is your DataFrame and you have 'stemmed_text' and 'label_angka' columns
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df['stemmed_text'], df['label_angka'], test_size=0.2, random_state=42)

# Initialize tokenizer
model_name = 'indobenchmark/indobert-base-p1'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize the text
def tokenize_data(text):
    return tokenizer(
        text.tolist(),
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='tf'
    )

# Tokenize train and test data
X_train_tokens = tokenize_data(X_train)
X_test_tokens = tokenize_data(X_test)

# Convert tokenized outputs to arrays
X_train_input_ids = X_train_tokens['input_ids'].numpy()
X_train_attention_mask = X_train_tokens['attention_mask'].numpy()
X_test_input_ids = X_test_tokens['input_ids'].numpy()
X_test_attention_mask = X_test_tokens['attention_mask'].numpy()

# Handle imbalanced data with SMOTE
X_train_combined = np.concatenate((X_train_input_ids, X_train_attention_mask), axis=1)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_combined, y_train)

# Split the combined array back into input_ids and attention_mask
X_train_input_ids_res = X_train_res[:, :128]
X_train_attention_mask_res = X_train_res[:, 128:]

# Load the BERT model
bert_model = TFBertModel.from_pretrained(model_name)

# Define the model
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

bert_outputs = bert_model({'input_ids': input_ids, 'attention_mask': attention_mask})
pooled_output = bert_outputs[1]  # BERT model returns pooled output at index 1

# Add a dropout layer and a dense layer for classification
dropout = tf.keras.layers.Dropout(0.3)(pooled_output)
output = tf.keras.layers.Dense(8, activation='softmax')(dropout)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model with sparse categorical crossentropy loss and accuracy metric
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Prepare data for training
train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': X_train_input_ids_res,
    'attention_mask': X_train_attention_mask_res
}, y_train_res)).shuffle(len(X_train_input_ids_res)).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': X_test_input_ids,
    'attention_mask': X_test_attention_mask
}, y_test)).batch(32)

# Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=3)

# Evaluate the model
predictions = model.predict(test_dataset)
predicted_labels = np.argmax(predictions, axis=1)
print(classification_report(y_test, predicted_labels))
print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test, predicted_labels)}")

Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from transformers import RobertaTokenizer
import numpy as np
import tensorflow as tf
from transformers import TFRobertaModel
from sklearn.metrics import classification_report, balanced_accuracy_score



# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df['stemmed_text'], df['label_angka'], test_size=0.2, random_state=42)

# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained('flax-community/indonesian-roberta-base')

# Tokenize the text
def tokenize_data(text):
    return tokenizer(
        text.tolist(),
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='np'
    )

# Tokenize train and test data
X_train_tokens = tokenize_data(X_train)
X_test_tokens = tokenize_data(X_test)

# Convert tokenized outputs to arrays
X_train_input_ids = X_train_tokens['input_ids']
X_train_attention_mask = X_train_tokens['attention_mask']
X_test_input_ids = X_test_tokens['input_ids']
X_test_attention_mask = X_test_tokens['attention_mask']

# Concatenate input_ids and attention_mask for SMOTE
X_train_combined = np.hstack((X_train_input_ids, X_train_attention_mask))

# Handle imbalanced data with SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_combined, y_train)

# Split the combined array back into input_ids and attention_mask
X_train_input_ids_res = X_train_res[:, :128]
X_train_attention_mask_res = X_train_res[:, 128:]

# Load the RoBERTa model
roberta_model = TFRobertaModel.from_pretrained('flax-community/indonesian-roberta-base')

# Define the model
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

roberta_outputs = roberta_model(input_ids, attention_mask=attention_mask)
pooled_output = roberta_outputs[1]

# Add a dropout layer and a dense layer for classification
dropout = tf.keras.layers.Dropout(0.3)(pooled_output)
output = tf.keras.layers.Dense(8, activation='softmax')(dropout)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Prepare data for training
train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': X_train_input_ids_res,
    'attention_mask': X_train_attention_mask_res
}, y_train_res)).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices(({        
    'input_ids': X_test_input_ids,
    'attention_mask': X_test_attention_mask
}, y_test)).batch(16)

# Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=3)



Some layers from the model checkpoint at cahya/roberta-base-indonesian-522M were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at cahya/roberta-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.



Epoch 1/3




RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL InvalidArgumentError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\framework\errors_impl.py(284): __init__
  c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\eager\execute.py(53): quick_execute
  c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\eager\context.py(1500): call_function
  c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\eager\polymorphic_function\atomic_function.py(251): call_flat
  c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\eager\polymorphic_function\atomic_function.py(216): call_preflattened
  c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\eager\polym

In [ ]:

# Predict on test data
y_pred_probs = model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate classification report and balanced accuracy
print(classification_report(y_test, y_pred))
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f'Balanced Accuracy: {balanced_acc}')

54/54 [==============================] - 77s 1s/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       561
           1       0.00      0.00      0.00        83
           2       0.00      0.00      0.00        46
           3       0.00      0.00      0.00        62
           4       0.00      0.00      0.00        68
           5       0.00      0.00      0.00        24
           6       0.00      0.00      0.00         9
           7       0.00      1.00      0.00         2

    accuracy                           0.00       855
   macro avg       0.00      0.12      0.00       855
weighted avg       0.00      0.00      0.00       855

Balanced Accuracy: 0.125


c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [ ]:
y_test.value_counts()

label_angka
0    561
1     83
4     68
3     62
2     46
5     24
6      9
7      2
Name: count, dtype: int64

In [ ]:
y_pred

array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the text
def tokenize_data(text):
    return tokenizer(
        text.tolist(),
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors='tf'
    )

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\thebl\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

c:\Users\thebl\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
X_train_tokens = tokenize_data(X_train)
X_test_tokens = tokenize_data(X_test)

In [ ]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_tokens['input_ids'], y_train)

In [ ]:
import tensorflow as tf
from transformers import TFRobertaModel

# Load the RoBERTa model
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

# Define the model
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

roberta_outputs = roberta_model(input_ids, attention_mask=attention_mask)
pooled_output = roberta_outputs[1]

# Add a dropout layer and a dense layer for classification
dropout = tf.keras.layers.Dropout(0.3)(pooled_output)
output = tf.keras.layers.Dense(8, activation='softmax')(dropout)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Prepare data for training
train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': X_train_res,
    'attention_mask': X_train_tokens['attention_mask'][:len(X_train_res)]
}, y_train_res)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': X_test_tokens['input_ids'],
    'attention_mask': X_test_tokens['attention_mask']
}, y_test)).batch(16)

# Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=3)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

ValueError: Dimensions 3416 and 18464 are not compatible

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')